<a href="https://colab.research.google.com/github/bcalik94/Multimodal-Speech-Summarisation-/blob/main/Inference_of_SentenceBERT%2BMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=1a558ca0ede575a8520d247c8807a1067efa4b02ee8a96cff6fe1267d2e0e59f
  Stored in directory: /tmp/pip-ephem-wheel-cache-jes_za5u/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip install -q transformers  rouge-score
!pip install rouge

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 596 kB 77.2 MB/s 
     |████████████████████████████████| 6.6 MB 68.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q transformers  rouge-score sentence-transformers


     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 31.2 MB/s 


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from tqdm.notebook import tqdm
 
import pickle
from rouge import Rouge
ROUGE = Rouge()
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
 
nlp = spacy.load('en_core_web_lg')

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
 
import pandas as pd 
import transformers
from torch.utils.data import Dataset, DataLoader 

from transformers import AutoTokenizer, AutoModel
sentenc_model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"
tokenizer = AutoTokenizer.from_pretrained(sentenc_model_name)

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
F0semitone = ['MEAN:F0semitoneFrom27.5Hz_sma3nz_amean',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
 'MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope']



F0semitone += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), F0semitone)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), F0semitone))
F0semitone.remove('MIN:F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope')
F0semitone.remove('MIN:F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope')
F0semitone.remove('MIN:F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope')



loudness = ['MEAN:loudness_sma3_amean',
 'MEAN:loudness_sma3_stddevNorm',
 'MEAN:loudness_sma3_percentile20.0',
 'MEAN:loudness_sma3_percentile50.0',
 'MEAN:loudness_sma3_percentile80.0',
 'MEAN:loudness_sma3_pctlrange0-2',
 'MEAN:loudness_sma3_meanRisingSlope',
 'MEAN:loudness_sma3_stddevRisingSlope',
 'MEAN:loudness_sma3_meanFallingSlope',
 'MEAN:loudness_sma3_stddevFallingSlope', 
 'MEAN:loudnessPeaksPerSec']

loudness += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), loudness)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), loudness))


spectral_Flux = ['MEAN:spectralFlux_sma3_amean',
 'MEAN:spectralFlux_sma3_stddevNorm', 
 'MEAN:spectralFluxV_sma3nz_amean',
 'MEAN:spectralFluxV_sma3nz_stddevNorm', 
 'MEAN:spectralFluxUV_sma3nz_amean']

spectral_Flux += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), spectral_Flux)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), spectral_Flux))

mfcc = ['MEAN:mfcc1_sma3_amean',
 'MEAN:mfcc1_sma3_stddevNorm',
 'MEAN:mfcc2_sma3_amean',
 'MEAN:mfcc2_sma3_stddevNorm',
 'MEAN:mfcc3_sma3_amean',
 'MEAN:mfcc3_sma3_stddevNorm',
 'MEAN:mfcc4_sma3_amean',
 'MEAN:mfcc4_sma3_stddevNorm',
 'MEAN:mfcc1V_sma3nz_amean',
 'MEAN:mfcc1V_sma3nz_stddevNorm',
 'MEAN:mfcc2V_sma3nz_amean',
 'MEAN:mfcc2V_sma3nz_stddevNorm',
 'MEAN:mfcc3V_sma3nz_amean',
 'MEAN:mfcc3V_sma3nz_stddevNorm',
 'MEAN:mfcc4V_sma3nz_amean',
 'MEAN:mfcc4V_sma3nz_stddevNorm']

mfcc += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), mfcc)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), mfcc))


jitter = ['MEAN:jitterLocal_sma3nz_amean',
 'MEAN:jitterLocal_sma3nz_stddevNorm']

jitter += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), jitter)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), jitter))


shimmer = ['MEAN:shimmerLocaldB_sma3nz_amean',
 'MEAN:shimmerLocaldB_sma3nz_stddevNorm']

shimmer += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), shimmer)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), shimmer))


HNR = ['MEAN:HNRdBACF_sma3nz_amean',
 'MEAN:HNRdBACF_sma3nz_stddevNorm']

HNR += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), HNR)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), HNR))


H1_A3 = ['MEAN:logRelF0-H1-H2_sma3nz_amean',
 'MEAN:logRelF0-H1-H2_sma3nz_stddevNorm',
 'MEAN:logRelF0-H1-A3_sma3nz_amean',
 'MEAN:logRelF0-H1-A3_sma3nz_stddevNorm']

H1_A3 += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), H1_A3)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), H1_A3))


F1 = ['MEAN:F1frequency_sma3nz_stddevNorm',
 'MEAN:F1bandwidth_sma3nz_amean',
 'MEAN:F1bandwidth_sma3nz_stddevNorm',
 'MEAN:F1amplitudeLogRelF0_sma3nz_amean',
 'MEAN:F1amplitudeLogRelF0_sma3nz_stddevNorm']

F1 += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), F1)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), F1))


F2 = ['MEAN:F2frequency_sma3nz_amean',
 'MEAN:F2frequency_sma3nz_stddevNorm',
 'MEAN:F2bandwidth_sma3nz_amean',
 'MEAN:F2bandwidth_sma3nz_stddevNorm',
 'MEAN:F2amplitudeLogRelF0_sma3nz_amean',
 'MEAN:F2amplitudeLogRelF0_sma3nz_stddevNorm']

F2 += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), F2)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), F2))


F3 = ['MEAN:F3frequency_sma3nz_amean',
 'MEAN:F3frequency_sma3nz_stddevNorm',
 'MEAN:F3bandwidth_sma3nz_amean',
 'MEAN:F3bandwidth_sma3nz_stddevNorm',
 'MEAN:F3amplitudeLogRelF0_sma3nz_amean',
 'MEAN:F3amplitudeLogRelF0_sma3nz_stddevNorm']

F3 += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'),F3)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), F3))


alphaRatio = ['MEAN:alphaRatioV_sma3nz_amean',
 'MEAN:alphaRatioV_sma3nz_stddevNorm',
 'MEAN:alphaRatioUV_sma3nz_amean']

alphaRatio  += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), alphaRatio )) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), alphaRatio ))

hammarberg = ['MEAN:hammarbergIndexV_sma3nz_amean',
 'MEAN:hammarbergIndexV_sma3nz_stddevNorm', 
 'MEAN:hammarbergIndexUV_sma3nz_amean']

hammarberg += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), hammarberg)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), hammarberg))


slopeV = ['MEAN:slopeV0-500_sma3nz_amean',
 'MEAN:slopeV0-500_sma3nz_stddevNorm',
 'MEAN:slopeV500-1500_sma3nz_amean',
 'MEAN:slopeV500-1500_sma3nz_stddevNorm', 
 'MEAN:slopeUV0-500_sma3nz_amean',
 'MEAN:slopeUV500-1500_sma3nz_amean']

slopeV  += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), slopeV )) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), slopeV ))

Segments = ['MEAN:VoicedSegmentsPerSec',
 'MEAN:MeanVoicedSegmentLengthSec',
 'MEAN:StddevVoicedSegmentLengthSec',
 'MEAN:MeanUnvoicedSegmentLength',
 'MEAN:StddevUnvoicedSegmentLength',
 'MEAN:equivalentSoundLevel_dBp']

Segments += list(map(lambda x: str(x).replace('MEAN:', 'MIN:'), Segments)) + list(map(lambda x: str(x).replace('MEAN:', 'MAX:'), Segments))
#Remove features which have 0 stds
Segments.remove('MIN:VoicedSegmentsPerSec')
Segments.remove('MIN:StddevVoicedSegmentLengthSec')
Segments.remove('MIN:StddevUnvoicedSegmentLength')
Segments.remove('MIN:MeanUnvoicedSegmentLength')
Segments.remove('MAX:MeanVoicedSegmentLengthSec')
Segments.remove('MAX:StddevUnvoicedSegmentLength')
Segments.remove('MEAN:StddevUnvoicedSegmentLength')



LLDs ={'F0semitone':F0semitone, 'loudness': loudness, 'spectral_Flux': spectral_Flux, 'mfcc': mfcc,'jitter': jitter,'shimmer': shimmer,'HNR':HNR ,'H1_A3': H1_A3, 'F1': F1, 'F2': F2,'F3' :F3,'alphaRatio': alphaRatio, 'hammarberg': hammarberg, 'slopeV': slopeV, 'Segments': Segments  }

In [ ]:

#Selected acoustic features for the model
acoustic_cols = ['F0semitone', 'loudness', 'spectral_Flux', 'mfcc', 'jitter', 'shimmer', 'HNR', 'H1_A3', 'F1', 'F2', 'F3', 'alphaRatio', 'hammarberg', 'slopeV', 'Segments']

In [ ]:
pickle_off = open ("//content/drive/MyDrive/data/ZNormedperSpeaker_L2_LLD_LDA_2000.txt", "rb")
LLD_LDA_df = pickle.load(pickle_off).sort_values('sentence_id').reset_index(drop = True)

sents_df = LLD_LDA_df.dropna().iloc[:-95,:]


#Split the dataset to train, val and test

#train_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in train_ids)
#val_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in valid_ids)
train_df = sents_df[sents_df['doc_id'].isin(test_ids)==False]
#test_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in test_ids)
test_df = sents_df[sents_df['doc_id'].isin(test_ids)]

print('train-val-test')
print(len(train_df), len(test_df))
print( "Train,val, test shape ", train_df.shape,test_df.shape)
print("Total number of episodes in all sets:" + str(len(train_df.groupby('episode_id'))))
print("Total number of episodes in valid sets:" + str(len(test_df.groupby('episode_id'))))
train_df


train-val-test
180312 21188
Train,val, test shape  (180312, 26) (21188, 26)
Total number of episodes in all sets:1797
Total number of episodes in valid sets:200


,sentence_id,doc_id,show_name,episode_id,Description,Transcript,speakerTag,sentence,rouge_score,label_range_perdoc,...,shimmer,HNR,H1_A3,F1,F2,F3,alphaRatio,hammarberg,slopeV,Segments
0,193,2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,presenting 'the very good podcast' from atlant...,I love the idea of Family Worship. Where can ...,2,Where can I go from your spirit?,0.043478,"(-0.001, 0.114]",...,-0.660877,-0.798128,-0.267261,-0.670600,-0.252395,-0.084075,-0.814404,-0.347449,-0.017098,-0.831912
1,194,2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,presenting 'the very good podcast' from atlant...,I love the idea of Family Worship. Where can ...,1,It's in our DNA well said Welcome to the very ...,0.214286,"(0.114, 0.297]",...,0.389356,-1.254771,-0.147058,-1.311794,-1.016248,-0.942490,-0.751163,-0.435297,1.529288,-0.222247
2,196,2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,presenting 'the very good podcast' from atlant...,I love the idea of Family Worship. Where can ...,1,I'm the director of communications for Atlanta...,0.2,"(0.114, 0.297]",...,-0.711160,-0.939281,-0.658271,-1.081243,-1.209267,-0.957355,-1.049705,-1.098483,-0.604528,-0.325575
3,197,2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,presenting 'the very good podcast' from atlant...,I love the idea of Family Worship. Where can ...,1,"If you're listening, you're already aware that...",0.067797,"(-0.001, 0.114]",...,-0.866658,-0.405215,-1.475857,-0.912256,-1.077218,-0.508776,-1.297199,-0.398153,-0.678356,-0.167439
4,198,2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,presenting 'the very good podcast' from atlant...,I love the idea of Family Worship. Where can ...,1,Our goal is to give you dear listener relevant...,0.075949,"(-0.001, 0.114]",...,-0.449453,-0.190128,0.932311,0.639453,0.888973,0.044059,0.063788,0.776748,0.430646,0.380527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201495,836832,3376,fey-sci and friends,spotify:episode:3URF7VVSis1hlobvBVGTbD,what a ridiculous and utterly obsolete questio...,"Hello, here. We are again to podcast Chris me...",2,"Yeah, and we still don't know how they were el...",0.138889,"(0.125, 0.18]",...,0.306729,-0.071032,0.738060,0.751846,0.573096,0.044388,1.190607,1.089764,0.959769,1.488234
201496,836854,3376,fey-sci and friends,spotify:episode:3URF7VVSis1hlobvBVGTbD,what a ridiculous and utterly obsolete questio...,"Hello, here. We are again to podcast Chris me...",2,"Priest tell me so yeah, the priests have to go...",0.07874,"(-0.001, 0.125]",...,-0.320859,-0.757922,1.118417,1.310367,0.852547,0.979421,0.894397,1.033240,1.098340,1.176957
201497,836861,3376,fey-sci and friends,spotify:episode:3URF7VVSis1hlobvBVGTbD,what a ridiculous and utterly obsolete questio...,"Hello, here. We are again to podcast Chris me...",2,Can you dial it back just a bit?,0.04,"(-0.001, 0.125]",...,-1.233475,-1.099726,-0.210735,-1.679530,-1.121849,-1.570689,-0.461219,-1.138795,-1.779491,-1.332234
201498,836866,3376,fey-sci and friends,spotify:episode:3URF7VVSis1hlobvBVGTbD,what a ridiculous and utterly obsolete questio...,"Hello, here. We are again to podcast Chris me...",2,So you saying if you would at the perfect we d...,0.180451,"(0.125, 0.18]",...,-0.008578,0.923152,0.291233,0.474505,0.327082,0.413231,0.814839,1.250667,0.557692,1.250369


In [ ]:
print(test_ids)

[1199, 1173, 765, 656, 1604, 2260, 42, 1702, 794, 1466, 6, 2600, 165, 471, 2854, 3074, 2559, 1470, 9, 2880, 2549, 2752, 23, 3342, 2963, 2671, 2819, 2378, 27, 566, 747, 3244, 2961, 1070, 2371, 373, 667, 2942, 1442, 3197, 177, 3293, 299, 2787, 2915, 3199, 827, 847, 100, 1253, 415, 3312, 793, 19, 1676, 2936, 1346, 575, 1308, 797, 1612, 3195, 921, 1276, 2270, 244, 131, 3154, 3118, 1508, 1190, 3298, 2896, 2461, 1263, 1471, 1342, 931, 505, 3305, 1029, 2034, 289, 3271, 151, 1092, 3158, 3167, 2983, 3322, 3110, 3200, 3114, 2921, 2431, 1035, 569, 499, 2334, 3327, 837, 833, 890, 2297, 654, 939, 2560, 431, 675, 3287, 376, 2602, 2672, 1251, 559, 2298, 789, 64, 1265, 492, 2953, 1191, 1765, 1428, 1332, 3086, 954, 2848, 2163, 411, 2673, 1282, 370, 1158, 313, 74, 2948, 1338, 329, 1219, 3329, 1175, 2857, 2950, 812, 841, 2335, 715, 778, 3071, 634, 170, 2871, 2068, 2191, 1269, 347, 208, 359, 3060, 1217, 2758, 3155, 288, 900, 730, 2990, 902, 3235, 2524, 906, 3356, 2694, 3065, 238, 1319, 3187, 281, 3174, 31

In [ ]:
pickle_off = open ("/content/drive/MyDrive/Yedek/allsents_df_4197episodes.txt", "rb")
selected_df = pickle.load(pickle_off)

import random

deneme_df = selected_df[selected_df['doc_id'].isin(test_ids)]

all_cols = [col for col in deneme_df if col.startswith('MAX') or col.startswith('MEAN') or col.startswith('MIN') ]
described= deneme_df.groupby(['doc_id','speakerTag'])[all_cols].agg([ 'std']).reset_index(level=[0,1]).rename(columns = {0:'doc_id'})

toremove = []
for column_name in described[all_cols].columns:
    column = described[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    print('Count of zeros in column ', column_name, ' is : ', count)
    if count >0:
      toremove.append(column_name[0])

corpus = deneme_df.drop(toremove, axis = 1)
all_cols = [col for col in corpus if col.startswith('MAX') or col.startswith('MEAN') or col.startswith('MIN') ]


temp_df = pd.DataFrame()
temp_df = corpus[all_cols]
temp_df['doc_id'] = corpus['doc_id'] 
temp_df['speakerTag'] = corpus['speakerTag'] 

groups = temp_df.groupby(['doc_id','speakerTag'])
mean, std = groups.transform("mean"), groups.transform("std")
ZnormalizedperSpeaker_subdf = (temp_df[all_cols] - mean) / std

ZNormedperSpeaker = corpus.copy()
ZNormedperSpeaker[all_cols] = ZnormalizedperSpeaker_subdf

ZNormedperSpeaker = ZNormedperSpeaker.dropna()
from sklearn import preprocessing
ZNormedperSpeaker_L2 = ZNormedperSpeaker.copy().reset_index(drop=True)
ZNormedperSpeaker_L2[all_cols] = pd.DataFrame(preprocessing.normalize(ZNormedperSpeaker[all_cols].values, norm='l2'), columns = all_cols)


Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_amean', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevNorm', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile20.0', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile50.0', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_percentile80.0', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope', 'std')  is :  0
Count of zeros in column  ('MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope', 'std')  is :  0
Count of zeros in column  ('MEAN:loudness_sma3_amean', 'std')  is

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
toremove

['MEAN:StddevUnvoicedSegmentLength',
 'MIN:F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'MIN:hammarbergIndexUV_sma3nz_amean',
 'MIN:spectralFluxUV_sma3nz_amean',
 'MIN:loudnessPeaksPerSec',
 'MIN:VoicedSegmentsPerSec',
 'MIN:StddevVoicedSegmentLengthSec',
 'MIN:MeanUnvoicedSegmentLength',
 'MIN:StddevUnvoicedSegmentLength',
 'MAX:alphaRatioUV_sma3nz_amean',
 'MAX:slopeUV0-500_sma3nz_amean',
 'MAX:slopeUV500-1500_sma3nz_amean',
 'MAX:loudnessPeaksPerSec',
 'MAX:VoicedSegmentsPerSec',
 'MAX:MeanVoicedSegmentLengthSec',
 'MAX:StddevUnvoicedSegmentLength']

In [ ]:

hammarberg.remove('MIN:hammarbergIndexUV_sma3nz_amean')


#F0semitone.remove('MIN:F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope')
slopeV.remove('MIN:slopeUV0-500_sma3nz_amean')
slopeV.remove('MAX:slopeUV0-500_sma3nz_amean')
spectral_Flux.remove('MIN:spectralFluxUV_sma3nz_amean')
loudness.remove('MIN:loudnessPeaksPerSec')
loudness.remove('MAX:loudnessPeaksPerSec')
Segments.remove('MIN:VoicedSegmentsPerSec')
slopeV.remove('MAX:slopeUV500-1500_sma3nz_amean')
Segments.remove('MAX:VoicedSegmentsPerSec')
#Segments.remove('MAX:MeanVoicedSegmentLengthSec')
#Segments.remove('MAX:MeanUnvoicedSegmentLength')
Segments.remove('MAX:StddevUnvoicedSegmentLength')
alphaRatio.remove('MAX:alphaRatioUV_sma3nz_amean')
'MIN:hammarbergIndexUV_sma3nz_amean',



In [ ]:
#Find LDA components of combined LLD features
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from itertools import product
import itertools

def LDA_for_LLDs(df): 

  LLD_scores = {}
  for LLD in LLDs.keys():
    print(LLD)
    #print(LLDs[LLD])
    X = df[LLDs[LLD]].values
    y = df['label_score']
    #Fit the LDA model
    lda = LinearDiscriminantAnalysis(n_components=1)
    #evaluate model
    transformed = lda.fit(X, y).transform(X)
    #print(LLD)
    #LLD_scores.append({str(LLD) : transformed.T})
    case = {str(LLD) : transformed.T.flatten()}
    LLD_scores.update(case)

  LLD_LDA = pd.DataFrame.from_dict(LLD_scores)

  LLD_LDA['y'] = df.reset_index(drop = True)['label_score']

  return LLD_LDA

LLDs ={'F0semitone':F0semitone, 'loudness': loudness, 'spectral_Flux': spectral_Flux, 'mfcc': mfcc,'jitter': jitter,'shimmer': shimmer,'HNR':HNR ,'H1_A3': H1_A3, 'F1': F1, 'F2': F2,'F3' :F3,'alphaRatio': alphaRatio, 'hammarberg': hammarberg, 'slopeV': slopeV, 'Segments': Segments  }
other_cols = ['sentence_id',
 'doc_id',
 'show_name',
 'episode_id',
 'Description',
 'Transcript',
 'speakerTag',
 'sentence',
  'rouge_score',
 'label_range_perdoc',
 'label_score'] 


ZNormedperSpeaker_L2_LLD_LDA = LDA_for_LLDs(ZNormedperSpeaker_L2)

LLD_LDA_ZNormedperSpeaker_L2= pd.concat([ZNormedperSpeaker_L2[other_cols],ZNormedperSpeaker_L2_LLD_LDA.drop('y', axis = 1)], axis =1)

test_df = LLD_LDA_ZNormedperSpeaker_L2

test_df

F0semitone
loudness
spectral_Flux
mfcc
jitter
shimmer
HNR
H1_A3
F1
F2
F3
alphaRatio
hammarberg
slopeV
Segments


,sentence_id,doc_id,show_name,episode_id,Description,Transcript,speakerTag,sentence,rouge_score,label_range_perdoc,...,shimmer,HNR,H1_A3,F1,F2,F3,alphaRatio,hammarberg,slopeV,Segments
0,1757,6,mental dumbells,spotify:episode:4G81Xg7If2oKsXGItBdauk,"sup fam, in this episode we talk about how we ...","goes Shut up, farewell one of the best inven...",2,"goes Shut up, farewell one of the best inventi...",0.060914,"(0.0501, 0.0865]",...,1.622310,1.487490,2.661210,1.691507,1.405772,1.209748,1.324672,0.853892,2.537660,1.340727
1,1758,6,mental dumbells,spotify:episode:4G81Xg7If2oKsXGItBdauk,"sup fam, in this episode we talk about how we ...","goes Shut up, farewell one of the best inven...",2,Ladies also left in the correct.,0.022472,"(-0.001, 0.0501]",...,-1.447889,-1.374636,-1.513054,-1.200358,-0.833271,-1.733470,-1.385859,-1.240755,-2.009899,-2.200773
2,1759,6,mental dumbells,spotify:episode:4G81Xg7If2oKsXGItBdauk,"sup fam, in this episode we talk about how we ...","goes Shut up, farewell one of the best inven...",2,"Correct, correct the situation a lot of the we...",0.033149,"(-0.001, 0.0501]",...,0.937415,0.587734,-1.088687,1.188816,0.929635,0.059394,-1.365701,-1.139769,-1.025446,-0.140885
3,1760,6,mental dumbells,spotify:episode:4G81Xg7If2oKsXGItBdauk,"sup fam, in this episode we talk about how we ...","goes Shut up, farewell one of the best inven...",3,Yeah a lot number got coffee.,0,"(-0.001, 0.0501]",...,-0.821560,0.650974,-1.014740,-0.608045,-0.290030,-0.817549,-0.335157,-0.348094,0.212604,0.378271
4,1761,6,mental dumbells,spotify:episode:4G81Xg7If2oKsXGItBdauk,"sup fam, in this episode we talk about how we ...","goes Shut up, farewell one of the best inven...",3,Shucks thing really but I think what we really...,0.010753,"(-0.001, 0.0501]",...,-1.137787,-0.601890,0.091838,0.551517,0.393273,0.539508,0.224156,0.860622,1.311577,0.874956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70970,830127,3356,a very nostalgic christmas,spotify:episode:0yyYhvKj4L6UbAAm2WFWkP,"hosts kay montgomery, princess carrol, bethani...","Hey guys, welcome to the Christmas 24/7. This...",3,That's Bethany with an ie awesome.,0.030303,"(-0.001, 0.0788]",...,0.211427,-0.737694,-0.557030,-0.239572,0.140491,-0.680333,-0.668794,-0.970563,-0.738029,0.507019
70971,830128,3356,a very nostalgic christmas,spotify:episode:0yyYhvKj4L6UbAAm2WFWkP,"hosts kay montgomery, princess carrol, bethani...","Hey guys, welcome to the Christmas 24/7. This...",3,So we'll see you guys next.,0.030303,"(-0.001, 0.0788]",...,-0.917262,-1.450698,-0.635851,-0.949999,-1.099666,-1.006432,-1.118654,-0.766911,-0.954010,-1.284491
70972,830129,3356,a very nostalgic christmas,spotify:episode:0yyYhvKj4L6UbAAm2WFWkP,"hosts kay montgomery, princess carrol, bethani...","Hey guys, welcome to the Christmas 24/7. This...",3,Last week thanks for tuning in.,0.045802,"(-0.001, 0.0788]",...,-1.007858,-0.661993,-1.547476,-1.276222,-1.328470,-1.275191,-1.330379,-0.928083,-1.160024,-0.636897
70973,830130,3356,a very nostalgic christmas,spotify:episode:0yyYhvKj4L6UbAAm2WFWkP,"hosts kay montgomery, princess carrol, bethani...","Hey guys, welcome to the Christmas 24/7. This...",3,Merry Christmas Merry Christmas Chief Spirit O...,0.098592,"(0.0788, 0.15]",...,0.354870,1.995061,1.061973,1.784444,1.044818,0.961686,2.219961,1.894950,1.094633,1.915744


In [ ]:
test_df = test_df[test_df['show_name'].isin(['marvel tv weekly - a marvel fan podcast', 'trev_talks', 'the late kick with josh pate','english speeches | learn english', 'servant after show podcast', 'the periodic bagel', 'vegains podcast', 'bbs are trash','the people engineer podcast' ]) == False ]

We need to normalize the acoustic values per episode, per speaker and globally

In [ ]:

#train_df.groupby('doc_id').agg(['mean', 'min', 'max', 'count', 'median' ])

#pd.DataFrame(train_df.groupby('doc_id')[mean_cols].agg([ 'min', 'max'])).T.reset_index(level=[0,1])

#train_df[all_acoustic_cols] = train_df.groupby('show_name')[all_acoustic_cols].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

#test_df[all_acoustic_cols] = test_df.groupby('show_name')[all_acoustic_cols].transform(lambda x: (x - x.min()) / (x.max() - x.min()))


# **Create a Data Loader Class**
Create a dataloader class that yields sentences and documentss and labels.

Setting hyperparameters for the model

In [ ]:

num_extra_dims = len(acoustic_cols)
print_n_steps = 1000
EPOCHS = 5
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 4
LEARNING_RATE = 2e-05 
acc_step_holder, loss_step_holder = [], []

model_name="sentence-transformers/paraphrase-MiniLM-L3-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
# Create a Data Loader Class
class SpotifyPodcastData(Dataset):
    def __init__(self, dataframe, tokenizer, acoustic_cols, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.acoustic_cols = acoustic_cols
        
    def __getitem__(self, index):
        sentence = str(self.data.iloc[index].sentence)
        sentence = " ".join(sentence.split())

        document = str(self.data.iloc[index].Transcript)
        document = " ".join(document.split())

        acoustics_info = np.stack(self.data[acoustic_cols].iloc[index].values).astype(np.float64) 

        inputs = self.tokenizer.batch_encode_plus(
            [sentence, document], 
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'sent_ids': torch.tensor(ids[0], dtype=torch.long),
            'doc_ids': torch.tensor(ids[1], dtype=torch.long),
            'sent_mask': torch.tensor(mask[0], dtype=torch.long),
            'doc_mask': torch.tensor(mask[1], dtype=torch.long),
            'acoustics' : torch.from_numpy(acoustics_info),
            'targets': torch.tensor([self.data.iloc[index].label_score], dtype=torch.float),
            'sentence': sentence
        } 
    
    def __len__(self):
        return self.len


testing_set = SpotifyPodcastData(test_df, tokenizer,acoustic_cols, MAX_LEN)


test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(testing_set, **test_params)





# **Build Model**
Build model based on sentence Bert pretrained models.

BERT-base consists of 12 transformer layers, each transformer layer takes in a list of token embeddings, and produces the same number of embeddings with the same hidden size (or dimensions) on the output. The output of the final transformer layer of the `[CLS]` token is used as the features of the sequence to feed a classifier.

The `transformers` library has the [`BertForSequenceClassification`](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification) class which is designed for classification tasks. However, we will create a new class so we can specify our own choice of classifiers.

Below we will create a BertClassifier class with a BERT model to extract the last hidden layer of the `[CLS]` token and a single-hidden-layer feed-forward neural network as our classifier.

In [ ]:
# get mean pooling for sentence bert models 
# ref https://www.sbert.net/examples/applications/computing-embeddings/README.html#sentence-embeddings-with-transformers
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
# Note that different sentence transformer models may have different in_feature sizes

#Dropout is only used during the training of a model and is not used when evaluating the skill of the model
class SentenceBertClass(torch.nn.Module):
    def __init__(self,model_name, num_extra_dims, in_features=384):
        """
        @param    bert: a BertModel object
        @param    MLP classifier: a torch.nn.Module MLP classifier for acoustic and BERT embeddings
        """
        super(SentenceBertClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(model_name)

        self.MLPclassifier = nn.Sequential(
            nn.Linear(in_features*3+num_extra_dims, 768),
            nn.ReLU(),
            nn.Dropout(0.15),
            #nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(0.15),
            torch.nn.Linear(768, 1),
            #nn.BatchNorm1d(1),
            torch.nn.Sigmoid()
        )

    def forward(self, sent_ids, doc_ids, sent_mask, doc_mask, acoustic_info):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """

        sent_output = self.l1(input_ids=sent_ids, attention_mask=sent_mask) 
        sentence_embeddings = mean_pooling(sent_output, sent_mask) 

        doc_output = self.l1(input_ids=doc_ids, attention_mask=doc_mask) 
        doc_embeddings = mean_pooling(doc_output, doc_mask)

        # elementwise product of sentence embs and doc embs
        combined_features = sentence_embeddings * doc_embeddings  

        # get concat of both features and elementwise product
        feat_cat = torch.cat((sentence_embeddings, doc_embeddings, combined_features, acoustic_info), dim=1)   
        
        output = self.MLPclassifier(feat_cat) 
        
        #output = self.classifier(feat_cat)
        return output

# Initialize the model

To fine-tune our Bert Classifier, we need to create an optimizer. The authors recommend following hyper-parameters:

Batch size: 8 or 16
Learning rate (Adam): 5e-5, 3e-5 or 2e-5
Number of epochs: 2, 3, 4

In [ ]:
#model path .. modify to location of trained model file 
model_path = "/content/drive/MyDrive/data/models/Multimodal_SentenceBERT_MLP_1Epoch_5KFold_1600_240522.pth"


from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

model = SentenceBertClass(model_name=model_name, num_extra_dims=len(acoustic_cols))
model.load_state_dict(torch.load(model_path, map_location=torch.device(device) ))
model.eval();

In [ ]:




def get_scores(model, testing_loader):
    model.eval()
    predictions, true_labels = [],[]
    all_logits = []
    target_list = []
    scores = []
    episode_sents = []

# For each batch in our test set...    
    for _, data in tqdm(enumerate(testing_loader, 0)): 
            # Load batch to GPU   
            sent_ids = data['sent_ids'].to(device, dtype = torch.long)
            doc_ids = data['doc_ids'].to(device, dtype = torch.long)
            sent_mask = data['sent_mask'].to(device, dtype = torch.long)
            doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
            targets = data['targets'].to(device, dtype = torch.float) 
            acoustic_info = data['acoustics'].to(device, dtype = torch.float)
            batch_sentences = data['sentence']
            # Compute logits 
            
             
            outputs = model(sent_ids, doc_ids, sent_mask, doc_mask,acoustic_info) 
            episode_sents.extend(batch_sentences)
            #print(batch_sentences)

            # Move logits and labels to CPU
            logits = outputs.detach().cpu().numpy()
            batch_targets = targets.to('cpu').numpy()


            # Store predictions and true labels
            predictions.append(logits)
            true_labels.append(batch_targets)
    
    return episode_sents,predictions

def flat_torch(listlist):

 outputs = []
 for ttorch in listlist:
    #print(ttorch.flatten().tolist())
    outputs.extend(ttorch.flatten().tolist())
 return outputs


 # Sort sentences to sum



def summarise(predictions, sents_list, top_k=4):

  # Combine the results across all batches.  
  flat_predictions = flat_torch(predictions)
  
  sent_pred_list = [{"sentence": sents_list[i], "score": flat_predictions[i], "index":i} for i in range(len(flat_predictions))]
  sorted_sentences = sorted(sent_pred_list, key=lambda k: k['score'], reverse=True) 

  sorted_result = sorted_sentences[:top_k] 
  sorted_result = sorted(sorted_result, key=lambda k: k['index']) 
  
  summary = [ x["sentence"] for x in sorted_result]
  summary = " ".join(summary)

  return summary, flat_predictions





#sents_list = episode_sents


In [ ]:
from rouge import Rouge
ROUGE = Rouge()

episodes = []
summaries = []
results = []
rouge_L = []
rouge_1 = []
rouge_2 = []


for doc in tqdm(test_df['doc_id'].unique().tolist()):

  episode_df = test_df[test_df['doc_id']== doc].reset_index(drop=True)
  testing_set = SpotifyPodcastData(episode_df, tokenizer,acoustic_cols, MAX_LEN)
  testing_loader = DataLoader(testing_set, **test_params)

  episode_sents, model_predictions = get_scores(model, testing_loader)
  summary, flat_predictions = summarise(model_predictions, episode_sents, top_k=3)
  rouge_per_episode = ROUGE.get_scores(summary,episode_df['Description'][0])
  
  summaries.append({'doc_id':episode_df['doc_id'][0], 'show_name':episode_df['show_name'][0],'Predicted': summary, 'Reference':episode_df['Description'].loc[0]})
  rouge_1.append(rouge_per_episode[0]['rouge-1'])
  rouge_2.append(rouge_per_episode[0]['rouge-2'])
  rouge_L.append(rouge_per_episode[0]['rouge-l'])
  #summaries.extend(summary)
summaries_df = pd.DataFrame(summaries)
final_output = pd.concat([ summaries_df,pd.DataFrame(rouge_L).add_prefix('ROUGE-L_'), pd.DataFrame(rouge_1).add_prefix('ROUGE-1_'), pd.DataFrame(rouge_2).add_prefix('ROUGE-2_')], ignore_index = False, axis =1, sort = False)

test_df.groupby(['doc_id']).size()

final_output

  0%|          | 0/187 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

,doc_id,show_name,Predicted,Reference,ROUGE-L_r,ROUGE-L_p,ROUGE-L_f,ROUGE-1_r,ROUGE-1_p,ROUGE-1_f,ROUGE-2_r,ROUGE-2_p,ROUGE-2_f
0,6,mental dumbells,Are you to use a square really go and leave a ...,"sup fam, in this episode we talk about how we ...",0.079545,0.122807,0.096552,0.090909,0.140351,0.110345,0.000000,0.000000,0.000000
1,9,mental dumbells,"Oh, don't be the one that I have to go all the...","in this episode, we invited aaroson's sister o...",0.111111,0.181818,0.137931,0.129630,0.212121,0.160920,0.000000,0.000000,0.000000
2,12,the great british shitcom,Hello and welcome to the last in the current s...,"in the final episode of the series, we decide ...",0.097561,0.184615,0.127660,0.097561,0.184615,0.127660,0.012821,0.022472,0.016327
3,19,not safe for worship,This is the first episode of the not safe for ...,the first episode!! cori and sean begin our po...,0.146341,0.107143,0.123711,0.170732,0.125000,0.144330,0.022222,0.012346,0.015873
4,23,forged by food,"If you haven't heard of it, it's called anchor...",in this episode we interview a long time frien...,0.233333,0.254545,0.243478,0.300000,0.327273,0.313043,0.074627,0.066667,0.070423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,3322,the why women kill podcast,Now you're mentioning that what was interestin...,"love, jealousy, who's killing who and infideli...",0.131579,0.107527,0.118343,0.197368,0.161290,0.177515,0.011111,0.006944,0.008547
183,3327,the fmcreators show,They get the Darby is called the Belfast big t...,welcome back to episode 10 of the fmcreators s...,0.131868,0.171429,0.149068,0.131868,0.171429,0.149068,0.014706,0.018349,0.016327
184,3329,the backpocket podcast,So what I want like people what that are liste...,offering your hand in service without an expec...,0.110294,0.156250,0.129310,0.161765,0.229167,0.189655,0.015385,0.019231,0.017094
185,3342,allsociology: take 1,"And I mean, this is really really General in m...",the third episode of the take 1 podcast. this ...,0.150943,0.066116,0.091954,0.169811,0.074380,0.103448,0.014925,0.005319,0.007843


In [ ]:
summaries_df

,doc_id,show_name,Predicted,Reference
0,6,mental dumbells,Are you to use a square really go and leave a ...,"sup fam, in this episode we talk about how we ..."
1,9,mental dumbells,"Oh, don't be the one that I have to go all the...","in this episode, we invited aaroson's sister o..."
2,12,the great british shitcom,Hello and welcome to the last in the current s...,"in the final episode of the series, we decide ..."
3,19,not safe for worship,This is the first episode of the not safe for ...,the first episode!! cori and sean begin our po...
4,23,forged by food,"If you haven't heard of it, it's called anchor...",in this episode we interview a long time frien...
...,...,...,...,...
182,3322,the why women kill podcast,Now you're mentioning that what was interestin...,"love, jealousy, who's killing who and infideli..."
183,3327,the fmcreators show,They get the Darby is called the Belfast big t...,welcome back to episode 10 of the fmcreators s...
184,3329,the backpocket podcast,So what I want like people what that are liste...,offering your hand in service without an expec...
185,3342,allsociology: take 1,"And I mean, this is really really General in m...",the third episode of the take 1 podcast. this ...


In [ ]:
final_output.groupby('doc_id').mean().mean()

ROUGE-L_r    0.190648
ROUGE-L_p    0.167993
ROUGE-L_f    0.162697
ROUGE-1_r    0.228240
ROUGE-1_p    0.200916
ROUGE-1_f    0.195457
ROUGE-2_r    0.050419
ROUGE-2_p    0.039914
ROUGE-2_f    0.038685
dtype: float64

In [ ]:
final_output['ROUGE-L_f'].mean()

0.16269676972219949